In [3]:
import os
import uuid
import paramiko
from scp import SCPClient
import json


# Mac 서버 정보
MAC_SERVER_HOST = "61.37.153.212"  # 서버 IP 주소
MAC_SERVER_USER = "yjtech_mac_machine"          # Mac 서버 사용자명
MAC_SERVER_KEY = "/home/yjtech/Desktop/LLM/key/id_rsa"  # Mac 서버 SSH 키 경로
REMOTE_SCRIPT_PATH = "/Users/yjtech_mac_machine/Desktop/yurim/llm/voice_to_text.py"  # Mac 서버의 스크립트 위치
REMOTE_TEMP_DIR = "/tmp"  # 서버 임시 디렉토리 (파일 저장 없이 작업 수행)

import json

def execute_on_mac(maf_idx, file_path):
    """
    Mac 서버로 SSH 연결하여 음성 파일 처리 작업 수행.
    결과를 stdout으로 받아 JSON 형태로 반환.
    """
    result = None
    ssh = paramiko.SSHClient()
    try:
        # SSH 클라이언트 설정
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        private_key = paramiko.RSAKey.from_private_key_file(MAC_SERVER_KEY)

        # SSH 연결
        print("Connecting to server...")
        ssh.connect(
            hostname=MAC_SERVER_HOST,
            username=MAC_SERVER_USER,
            pkey=private_key
        )
        print("Connected to server successfully!")

        # 파일 전송 (GCP → Mac)
        temp_file_name = os.path.basename(file_path)
        remote_temp_path = os.path.join(REMOTE_TEMP_DIR, temp_file_name)

        print(f"Uploading file: {file_path} to {remote_temp_path}")
        with SCPClient(ssh.get_transport()) as scp:
            scp.put(file_path, remote_temp_path)

        # 명령어 실행: 결과를 stdout으로 출력
        command = (
            f"export PATH=/opt/homebrew/bin:$PATH && "
            f"/opt/anaconda3/envs/venv-keyword/bin/python {REMOTE_SCRIPT_PATH} "
            f"--maf_idx {maf_idx} --file_path {remote_temp_path}"
        )
        print(f"Executing command: {command}")
        stdin, stdout, stderr = ssh.exec_command(command)
        stdout.channel.recv_exit_status()  # 명령어 실행 완료 대기

        # 서버 출력 결과 가져오기
        output = stdout.read().decode().strip()
        error = stderr.read().decode()

        if error:
            print(f"Error from server: {error}")
            raise Exception(f"Error during script execution: {error}")

        print("Received output from server:")
        print(output)


    except Exception as e:
        print(f"Unexpected Error: {e}")
    finally:
        ssh.close()

    return output


In [5]:
maf_idx = 1
file_path = '/home/yjtech/Desktop/LLM/sample_audio.webm'


voice_datas = {'maf_idx': maf_idx, 'file_path': file_path}
result = execute_on_mac(voice_datas['maf_idx'], voice_datas['file_path'])

print(f"Final result: {result}")

Connecting to server...
Connected to server successfully!
Uploading file: /home/yjtech/Desktop/LLM/sample_audio.webm to /tmp/sample_audio.webm
Executing command: export PATH=/opt/homebrew/bin:$PATH && /opt/anaconda3/envs/venv-keyword/bin/python /Users/yjtech_mac_machine/Desktop/yurim/llm/voice_to_text.py --maf_idx 1 --file_path /tmp/sample_audio.webm
Error from server: /opt/anaconda3/envs/venv-keyword/lib/python3.8/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed